In [ ]:

from openai import OpenAI, AsyncOpenAI
import statistics
import os
import pandas as pd
import requests
import os
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract
from collections import defaultdict
import base64
import os
import pickle
import scipy.stats as stats
import asyncio
os.environ["OPENAI_API_KEY"] = "#put your key here"

In [ ]:
def load_pickle(file_name: str):
    if os.path.exists(f'data/{file_name}'):
        with open(f'data/{file_name}', "rb") as f:
            return pickle.load(f)
    else:
        return defaultdict(list)

def save_pickle(file_name: str, data):
    with open(f'data/{file_name}', "wb") as f:
        pickle.dump(data, f)


In [ ]:
# Load CSV and remove rows where 'ticker' is NaN
df = pd.read_csv("earnings_reports.csv")
df = df[df["ticker"].notna()]
file_ids = load_pickle(f'ticker_file_id_map.pickle')


In [ ]:
df

## Utils

In [ ]:
def get_pre_earnings_price(row: pd.Series, quarter: str):
    if quarter == 'Q1':
        return row['Close - day of release']
    elif quarter == 'Q4_2024':
        return row['Close - day of release.1']
    else:
        return row['close_price_release_day']

def get_post_earnings_price_open(row: pd.Series, quarter: str):
    if quarter == 'Q1':
        return row['Open - next day']
    elif quarter == 'Q4_2024':
        return row['Open - next day.1']
    else:
        return row['nextday_open']

def get_post_earnings_price_close(row: pd.Series, quarter: str):
    if quarter == 'Q1':
        return row['Close - next day']
    elif quarter == 'Q4_2024':
        return row['Close - next day.1']
    else:
        return row['nextday_close']

def get_label(pre_price: float, post_price: float):
    if (abs(post_price - pre_price) / pre_price) < 0.01:
        return "flat"
    else:
        return "up" if post_price > pre_price else "down"

def get_ticker_id(ticker: str, quarter: str):
    return f"{quarter}_{ticker}".replace(":", "_")

def get_pre_sap_price(row: pd.Series, quarter: str):
    if quarter == 'Q1':
        return row['sap500_before_close.1']
    elif quarter == 'Q4_2024':
        return row['sap500_before_close.2']
    else:
        return row['sap500_before_close']

def get_post_sap_price(row: pd.Series, quarter: str, compares_to_close_price: bool):
    suffix = "close" if compares_to_close_price else "open"
    if quarter == 'Q1':
        return row[f'sap500_after_{suffix}.1']
    elif quarter == 'Q4_2024':
        return row[f'sap500_after_{suffix}.2']
    else:
        return row[f'sap500_after_{suffix}']

def get_analysts_estimates(row: pd.Series, quarter: str):
    if quarter == 'Q1':
        return row['Estimates.1']
    elif quarter == 'Q4_2024':
        return row['Estimates.2']
    else:
        return row['Estimates']

In [ ]:
def get_prompt(row: pd.Series, is_thinking: bool, includes_past_movements: bool, includes_sap_movement: bool, includes_analysts_expectations: bool, include_past_reports: bool, quarter: str, compares_to_close_price: bool):
    includes_past_movements_prompt = ""
    if includes_past_movements:
        q4_2024_pre_price = get_pre_earnings_price(row, 'Q4_2024')
        q4_2024_next_day = get_post_earnings_price_close(row, 'Q4_2024') if compares_to_close_price else get_post_earnings_price_open(row, 'Q4_2024')
        q4_2024_estimates = get_analysts_estimates(row, 'Q4_2024')
        if quarter == 'Q2':
            q1_pre_price = get_pre_earnings_price(row, 'Q1')
            q1_next_day = get_post_earnings_price_close(row, 'Q1') if compares_to_close_price else get_post_earnings_price_open(row, 'Q1')
            q1_estimates = get_analysts_estimates(row, 'Q1')
            includes_past_movements_prompt = f" and previous quarter (beginning of 2025) the movement was from {q1_pre_price} to {q1_next_day} based on estimates {q1_estimates}, end quarter of 2024 registered movement of {q4_2024_pre_price} to {q4_2024_next_day} based on estimates {q4_2024_estimates}"
        elif quarter == 'Q1':
            includes_past_movements_prompt = f" and previouse end quarter of 2024 registered movement of {q4_2024_pre_price} to {q4_2024_next_day} based on estimates {q4_2024_estimates}"
    
    thinking_prompt = ". But Firstly explain yourself briefly. " if is_thinking else ". Only output price."
    
    close_price = get_pre_earnings_price(row, quarter)
    next_prompt = "close price" if compares_to_close_price else "open price"
    sap_before_price = get_pre_sap_price(row, quarter)
    sap_after_price = get_post_sap_price(row, quarter, compares_to_close_price)
    estimates = f' and following analysts expectations outcomes \"{get_analysts_estimates(row, quarter)}\"' if includes_analysts_expectations else ""
    sap_movement_prompt = f", S&P500 movement before and after earnings release publish in the same period was from {sap_before_price} to {sap_after_price}" if includes_sap_movement else ""
    past_reports_prompt = f" and past reports for past market behavioural context" if include_past_reports else ""

    prompt = f"""
    You are a financial analyst.

    Question:
    Based on the attached earnings report{past_reports_prompt}:
    - Given the current close price before publishing of the release was {close_price:.2f}{sap_movement_prompt}{includes_past_movements_prompt}{estimates}, predict next {next_prompt}{thinking_prompt}
    Last sentence, on a new line, put a number (stock price) you estimate the stock to be on, nothing else.
    """
    return prompt

print(get_prompt(df.iloc[0], False, True, True, True, True, 'Q2', True))

In [ ]:
def extract_prediction(response, row: pd.Series, quarter: str):
    output = response.output_text.strip()
    # print("Raw model output:\n", output) 

    # Split on newlines
    lines = [line.strip() for line in output.splitlines() if line.strip()]  # removes empty strings
    #print("DEBUG lines", lines)

    # Extract values
    predicted_price = lines[-1].strip(" #*$")

    if ":" in predicted_price:
        predicted_price = predicted_price.split(":")[1].strip()
    if "**" in predicted_price:
        predicted_price = predicted_price.split("**")[1].strip()
        if "**" in predicted_price:
            predicted_price = predicted_price.split("**")[0].strip()
    if ")" in predicted_price:
        predicted_price = predicted_price.split(")")[1].strip()
    if "at" in predicted_price:
        predicted_price = predicted_price.split("at")[-1].strip()

    predicted_price = predicted_price.strip(' .*$KRWCHF').replace(",", "")

    result = float(predicted_price)
    pre_price = get_pre_earnings_price(row, quarter)
    direction = get_label(pre_price, result)
    #print("DEBUG prediction", predicted_price)
    return result, direction

In [ ]:
client = AsyncOpenAI()
async def get_avg_prediction(
    row: pd.Series, 
    is_thinking: bool, 
    includes_past_movements: bool, 
    include_past_reports: bool, 
    includes_sap_movement: bool,
    includes_analysts_expectations: bool,
    compares_to_close_price: bool,
    runs: int, 
    model: str, 
    quarter: str, 
    predictions: defaultdict,
    k: tuple
):
    prices = []
    directions = []
    ticker = row['ticker']
    ticker_id = get_ticker_id(ticker, quarter)
    last_price = get_pre_earnings_price(row, quarter)

    past_reports = []
    if include_past_reports and quarter == "Q2":
        past_reports.append({
            "type": "input_file",
            "file_id": file_ids[get_ticker_id(ticker, "Q1")],
        })
    if include_past_reports and quarter in ["Q2", "Q1"]:
        past_reports.append({
            "type": "input_file",
            "file_id": file_ids[get_ticker_id(ticker, "Q4_2024")],
        })
    past_reports = []

    retries = 3
    while len(predictions[k]) < runs:
        try:
            response = await client.responses.create(
                model=model,
                temperature=None if model.startswith('gpt-5') else 0.1,
                reasoning={"effort": "medium"} if (model.startswith('gpt-5') and is_thinking) else None,
                input=[{
                        "role": "user",
                        "content": [
                            {"type": "input_text", "text": get_prompt(row, is_thinking, includes_past_movements, includes_sap_movement, includes_analysts_expectations, include_past_reports, quarter, compares_to_close_price)},
                            {
                                "type": "input_file",
                                "file_id": file_ids[ticker_id],
                            },
                        ] + past_reports,
                    }]
            )
            predicted_price, direction = extract_prediction(response, row, quarter)
        except Exception as e:
            print(f"Error extracting prediction, retries left: {retries}, error: {e}")
            retries -= 1
            if retries == 0:
                raise e
            continue

        predictions[k].append((response.output_text.strip(), predicted_price, direction))

        if predicted_price is not None:
            prices.append(predicted_price)
        if direction:
            directions.append(direction)
        #print(f"Run {i+1}: {predicted_price}, {direction}")

    # Compute average and most common direction
    avg_price = statistics.mean(prices) if prices else None
    most_common_direction = max(set(directions), key=directions.count) if directions else None

    print("\n--- Final Results ---")
    print(f"Ticker: {ticker}, Quarter: {quarter}, Model: {model}, Is thinking: {is_thinking}, Includes past movements: {includes_past_movements}, Include past reports: {include_past_reports}, Includes sap movement: {includes_sap_movement}, Includes analysts expectations: {includes_analysts_expectations}, Compares to close price: {compares_to_close_price}")
    print(f"Average predicted price: {avg_price}, real post price {get_post_earnings_price_open(row, quarter)}, pre-price {last_price}, most common direction: {most_common_direction}")
    
    return avg_price, most_common_direction
test = defaultdict(list)
await get_avg_prediction(
    row=df.iloc[1],
    is_thinking=True,
    includes_past_movements=True,
    include_past_reports=True,
    includes_sap_movement=True,
    includes_analysts_expectations=True,
    compares_to_close_price=False,
    runs=1,
    model='gpt-4o-mini',
    quarter='Q2',
    predictions=test,
    k=('SOFI', 'Q2', 'gpt-4o-mini', True, False, False, False, False, False, 549.0, 592.08)
)
test

In [ ]:
def flatten_predictions(test_dict):
    # Flatten the test dict into one record per key, merging all predictions into one row per k
    records = []
    for key, pred_list in test_dict.items():
        # key is a tuple (ticker, model, is_thinking)
        record = {
            "ticker": key[0],
            "quarter": key[1],
            "model": key[2],
            "is_thinking": key[3],
            "includes_past_movements": key[4],
            "include_past_reports": key[5],
            "includes_sap_movement": key[6],
            "includes_analysts_expectations": key[7],
            "compares_to_close_price": key[8],
            "close_price_release_day": key[9],
            "nextday_price": key[10],
            "movement": get_label(key[9], key[10])
        }
        # Merge all predictions into columns for this record
        prices = []
        directions = []
        for i, tup in enumerate(pred_list, 1):
            record[f"text_{i}"] = tup[0]
            record[f"predicted_price_{i}"] = tup[1]
            record[f"direction_{i}"] = tup[2]
            if tup[1] is not None:
                prices.append(tup[1])
            if tup[2]:
                directions.append(tup[2])
        # Add average prediction, median prediction, majority direction
        record["avg_predicted_price"] = statistics.mean(prices) if prices else None
        record["median_predicted_price"] = statistics.median(prices) if prices else None
        record["majority_direction"] = max(set(directions), key=directions.count) if directions else None

        # Add % movement in real price
        record["real_price_movement_pct"] = (record["nextday_price"] - record["close_price_release_day"]) / record["close_price_release_day"] * 100
        record["avg_pred_vs_real_pct"] = abs((record["avg_predicted_price"] - record["nextday_price"]) / record["close_price_release_day"] * 100)
        record["median_pred_vs_real_pct"] = abs((record["median_predicted_price"] - record["nextday_price"]) / record["close_price_release_day"] * 100)

        records.append(record)

    df_out = pd.DataFrame(records)
    # Move 'median_predicted_price' after 'movement'
    cols = list(df_out.columns)
    cols.remove('median_predicted_price')
    movement_idx = cols.index('movement')
    cols.insert(movement_idx + 1, 'median_predicted_price')
    df_out = df_out[cols]
    return df_out

df_test = flatten_predictions(test)
df_test

In [ ]:
flat_df[["quarter", "ticker", "movement", "close_price_release_day", "nextday_price"]]

## Predict

In [ ]:
predictions = load_pickle('predictions.pickle')

runs = 3
is_thinking = True
includes_past_movements = True
include_past_reports = True
includes_sap_movement=True,
includes_analysts_expectations=True,
compares_to_close_price=False,

awaitables = []

for model in ['gpt-4o-mini', 'gpt-5.1', 'gpt-5']:
    for quarter in ['Q2', 'Q1', 'Q4_2024']:
        for i, row in df.iterrows():
            ticker = row['ticker']
            ticker_id = get_ticker_id(ticker, quarter)
            assert ticker_id in file_ids, f"Ticker not in file_ids {ticker_id}"

            k = (
                ticker,
                quarter,
                model,
                is_thinking,
                includes_past_movements,
                include_past_reports,
                includes_sap_movement,
                includes_analysts_expectations,
                compares_to_close_price,
                get_pre_earnings_price(row, quarter),
                get_post_earnings_price_close(row, quarter) if compares_to_close_price else get_post_earnings_price_open(row, quarter),
            )
            if k in predictions and len(predictions[k]) >= runs:
                continue
            print(f"Processing {ticker}, {quarter}, row {i+1} of {len(df)}")
            
            awaitables.append(
                get_avg_prediction(
                    row=row,
                    is_thinking=is_thinking,
                    includes_past_movements=includes_past_movements,
                    include_past_reports=include_past_reports,
                    includes_sap_movement=includes_sap_movement,
                    includes_analysts_expectations=includes_analysts_expectations,
                    compares_to_close_price=compares_to_close_price,
                    runs=runs,
                    model=model,
                    quarter=quarter,
                    predictions=predictions,
                    k=k
                )
            )

results = await asyncio.gather(*awaitables, return_exceptions=True)
for res in results:
    if isinstance(res, Exception):
        print(f"Coroutine raised an exception: {res}")

save_pickle('predictions.pickle', predictions)

Processing GOOG, Q2, row 1 of 40
Processing META, Q2, row 2 of 40
Processing MSFT, Q2, row 3 of 40
Processing AMZN, Q2, row 4 of 40
Processing AAPL, Q2, row 5 of 40
Processing LCID, Q2, row 6 of 40
Processing VALE, Q2, row 7 of 40
Processing SOFI, Q2, row 8 of 40
Processing TSLA, Q2, row 9 of 40
Processing INTC, Q2, row 10 of 40
Processing PLTR, Q2, row 11 of 40
Processing NVO, Q2, row 12 of 40
Processing AMD, Q2, row 13 of 40
Processing SOUN, Q2, row 14 of 40
Processing BBAI, Q2, row 15 of 40
Processing OPEN, Q2, row 16 of 40
Processing BA, Q2, row 17 of 40
Processing VKTX, Q2, row 18 of 40
Processing WK, Q2, row 19 of 40
Processing APPF, Q2, row 20 of 40
Processing IRTC, Q2, row 21 of 40
Processing BIO, Q2, row 22 of 40
Processing RDDT, Q2, row 23 of 40
Processing RKT, Q2, row 24 of 40
Processing FLR, Q2, row 25 of 40
Processing ENVX, Q2, row 26 of 40
Processing EMN, Q2, row 27 of 40
Processing RIOT, Q2, row 28 of 40
Processing COIN, Q2, row 29 of 40
Processing FNMA, Q2, row 30 of 40

In [ ]:
flat_df = flatten_predictions(predictions)
flat_df

In [249]:
# Calculate and display results



def calculate_results(
    model: str,
    is_thinking: bool,
    includes_past_movements: bool,
    include_past_reports: bool,
    includes_sap_movement: bool,
    includes_analysts_expectations: bool,
    compares_to_close_price: bool
):

    flat_df = flatten_predictions(predictions)

    filtered_df = flat_df[
        (flat_df["model"] == model) &
        (flat_df["is_thinking"] == is_thinking) &
        (flat_df["includes_past_movements"] == includes_past_movements) &
        (flat_df["include_past_reports"] == include_past_reports) &
        (flat_df["includes_sap_movement"] == includes_sap_movement) &
        (flat_df["includes_analysts_expectations"] == includes_analysts_expectations) &
        (flat_df["compares_to_close_price"] == compares_to_close_price)
    ]

    # Percentage of 'up' in movement
    up_count = (filtered_df["movement"] == "up").sum()
    total_count = filtered_df["movement"].notna().sum()
    percent_up = (up_count / total_count) * 100 if total_count > 0 else 0

    # Breakdown predictions
    majority_up_count = (filtered_df["majority_direction"] == "up").sum()
    majority_down_count = (filtered_df["majority_direction"] == "down").sum()
    majority_flat_count = (filtered_df["majority_direction"] == "flat").sum()
    majority_total_count = filtered_df["majority_direction"].notna().sum()
    percent_majority_up = (majority_up_count / majority_total_count) * 100
    percent_majority_down = (majority_down_count / majority_total_count) * 100
    percent_majority_flat = (majority_flat_count / majority_total_count) * 100

    # Percentage where movement and majority_direction match
    match_mask = (filtered_df["movement"].notna()) & (filtered_df["majority_direction"].notna())
    matches = (filtered_df["movement"][match_mask] == filtered_df["majority_direction"][match_mask]).sum()
    total_matches = match_mask.sum()
    percent_matches = (matches / total_matches) * 100 if total_matches > 0 else 0

    # Hypothesis test: is the proportion of matches statistically significantly different from random chance?
    p_null = 0.38  # e.g., random 50/50 guessing
    if total_matches > 0:
        p_result = stats.binomtest(matches, total_matches, p_null, alternative='two-sided')
        p_value = p_result.pvalue
        ci_low, ci_upp = stats.binom.interval(0.95, total_matches, matches/total_matches, loc=0)
        ci_low = (ci_low / total_matches) * 100 if total_matches > 0 else 0
        ci_upp = (ci_upp / total_matches) * 100 if total_matches > 0 else 0
    else:
        p_value = None
        ci_low, ci_upp = None, None

    print(f"Matches {matches} out of {total_matches} = {matches / total_matches * 100:.2f}%")
    print(f"Percentage of 'up' movements: {percent_up:.2f}%")
    print(f"Predictions counts: up {majority_up_count}, down {majority_down_count}, flat {majority_flat_count}")
    print(f"Percentage of predictions: up {percent_majority_up:.2f}% down {percent_majority_down:.2f}% flat {percent_majority_flat:.2f}%")
    print(f"Percentage where movement and majority_direction match: {percent_matches:.2f}%")
    if p_value is not None:
        print(f"p-value of match proportion vs 0.38: {p_value:.4f}")
        print(f"95% confidence interval for match percentage: [{ci_low:.2f}%, {ci_upp:.2f}%]")
    else:
        print("Not enough data for p-value/confidence interval calculation.")


    print()

    # Calculate and print the average absolute value of real_price_movement_pct
    avg_abs_real_price_movement = filtered_df["real_price_movement_pct"].abs().mean()
    print(f"Average absolute real_price_movement_pct: {avg_abs_real_price_movement:.2f}%")

    # Calculate and print the average absolute value of median_pred_vs_real_pct
    avg_abs_median_pred_vs_real = filtered_df["median_pred_vs_real_pct"].abs().mean()
    print(f"Average absolute median_pred_vs_real_pct: {avg_abs_median_pred_vs_real:.2f}%")

    # Calculate and print the average absolute value of avg_pred_vs_real_pct
    avg_abs_avg_pred_vs_real = filtered_df["avg_pred_vs_real_pct"].abs().mean()
    print(f"Average absolute avg_pred_vs_real_pct: {avg_abs_avg_pred_vs_real:.2f}%")

In [250]:
model = "gpt-5.1"
is_thinking = True
includes_past_movements = True
include_past_reports = True
includes_sap_movement=True,
includes_analysts_expectations=True,
compares_to_close_price=True,

calculate_results(model, is_thinking, includes_past_movements, include_past_reports, includes_sap_movement, includes_analysts_expectations, compares_to_close_price)

Matches 56 out of 120 = 46.67%
Percentage of 'up' movements: 36.67%
Predictions counts: up 73, down 43, flat 4
Percentage of predictions: up 60.83% down 35.83% flat 3.33%
Percentage where movement and majority_direction match: 46.67%
p-value of match proportion vs 0.38: 0.0596
95% confidence interval for match percentage: [37.50%, 55.83%]

Average absolute real_price_movement_pct: 7.87%
Average absolute median_pred_vs_real_pct: 8.71%
Average absolute avg_pred_vs_real_pct: 8.69%


In [252]:
model = "gpt-4o-mini"
is_thinking = True
includes_past_movements = True
include_past_reports = True
includes_sap_movement=True,
includes_analysts_expectations=True,
compares_to_close_price=True,

calculate_results(model, is_thinking, includes_past_movements, include_past_reports, includes_sap_movement, includes_analysts_expectations, compares_to_close_price)

Matches 54 out of 120 = 45.00%
Percentage of 'up' movements: 36.67%
Predictions counts: up 87, down 30, flat 3
Percentage of predictions: up 72.50% down 25.00% flat 2.50%
Percentage where movement and majority_direction match: 45.00%
p-value of match proportion vs 0.38: 0.1320
95% confidence interval for match percentage: [35.83%, 54.17%]

Average absolute real_price_movement_pct: 7.87%
Average absolute median_pred_vs_real_pct: 9.00%
Average absolute avg_pred_vs_real_pct: 9.11%
